In [1]:
import pandas as pd
import powerplantmatching as pm
import pypsa
import pycountry
import os 
import numpy as np

In [2]:
def prepare_capacity_table(capacity):
    
    capacity_matching = pd.Series(
        ["nuclear", "lignite", "coal", "gas", "oil", "other", "ROR", "ROR", "reservoir", "reservoir", "PHS",
         "reservoir (pumping)", "PHS (pumping)", "onwind", "offwind", "CSP", "solar", "biomass", "biomass", "battery"],
        capacity.index
    )

    capacity = capacity.groupby(capacity_matching, ).sum()
    
    pm_plants = pm.powerplants()
    gas_share = pd.DataFrame()
    gas_share["CCGT"] = pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", "CCGT",:].Capacity.div(
        pm_plants.groupby(["Fueltype", "Technology", "Country"]).sum(numeric_only=True).loc["Natural Gas", :].Capacity.groupby(level=1).sum(),
        fill_value=0
    )

    gas_share.index = [pycountry.countries.lookup(i).alpha_2 for i in gas_share.index]

    gas_share["OCGT"] = 1- gas_share["CCGT"]

    gas_share = gas_share.reindex(capacity.columns.str[:2]).fillna(gas_share.mean())
    gas_share.index = capacity.columns

    capacity = pd.concat([
        capacity.drop("gas"),
        gas_share.multiply(capacity.loc["gas"],axis=0).T
    ])
    
    return capacity


In [3]:
def build_capacity_table():

    capacity_years = pd.DataFrame()

    for sheet in plant_sheets.index:

        capacity_raw = pd.read_excel(excel_file, sheet, header=1, index_col=1).dropna(how="all", axis=1).iloc[:20]

        capacity_years[sheet[2:]] = prepare_capacity_table(capacity_raw).stack().astype(float)

    capacity_years.columns = capacity_years.columns.astype(int)
    capacity_years = capacity_years.reindex(years,axis=1)
    capacity_years.loc[distributed_resources, :] = capacity_years.loc[distributed_resources, :].interpolate(axis=1)
    

    return capacity_years.fillna(method="ffill", axis=1)[year].unstack(1)

In [4]:
def build_inflows(inflows):
    base_year_hydro = pd.Series(
        inflows.index.levels[4].astype(int), 
        inflows.index.levels[4]).subtract(year).abs().idxmin()

    inflows = inflows.loc[:, str(climate_year), :, :, base_year_hydro]

    inflow_grouper = pd.Series(["ROR", "PHS", "reservoir", "reservoir", "ROR"], inflows.index.levels[1])

    inflows = inflows.unstack(1).groupby(inflow_grouper, axis=1).sum().multiply(1e3) # conversion to MWh
    
    inflows = inflows.unstack(0).stack(0)
    inflows.index = [" ".join(i) for i in inflows.index]
    
    inflows = inflows.T
    
    inflows.index = snapshots
    
    return inflows

In [50]:
def add_existing_storage():  
 
    storage = capacity.loc[["PHS", "reservoir", "battery"]].unstack().reset_index().copy()
    storage.columns = ["bus", "carrier", "p_nom"]
    storage.index = storage.bus + " " + storage.carrier

    p_min_pu = capacity.loc[["PHS (pumping)", "reservoir (pumping)",]].div(capacity.loc[["PHS", "reservoir"]].add(1e-5).values)
    p_min_pu.index = ["PHS", "reservoir"]

    storage["p_min_pu"] = (
        p_min_pu.unstack()
        .reindex(pd.MultiIndex.from_arrays([storage.bus, storage.carrier]))
    ).values

    storage.loc[storage.carrier=="battery", "p_min_pu"] = -1

    storage = storage[storage.p_nom >0]

    storage_inflows = inflows[storage.loc[storage.carrier=="reservoir"].index]

    storage_capacity_raw = pd.read_excel(excel_file, sheet, index_col=1, header=28).dropna(how="all", axis=1)

    storage_capacity= storage_capacity_raw.groupby(
        ["ROR", "reservoir", "reservoir", "PHS", "battery"]
    ).sum()

    storage["max_hours"] = storage_capacity.unstack().reindex(
        pd.MultiIndex.from_arrays(
            [storage.bus, storage.carrier]  
        )
    ).div(
        storage.p_nom.values
    ).values

    map_storage = pd.Series( ["battery inverter", "PHS", "hydro"], ["battery", "PHS", "reservoir"])

    storage["efficiency_store"] = technology_data.loc[:, "efficiency", :].reindex(storage.carrier.map(map_storage)).value.values

    n.madd(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows,
        invest_status = "existing"
    )

In [6]:
def prepare_commodity_prices(commodity_prices):

    commodity_prices = commodity_prices_raw.iloc[:, 2:].groupby(commodity_prices.index).mean()
    commodity_prices = commodity_prices.groupby(["co2", "CCGT", "coal", "oil", "hydrogen", "oil", "lignite", "nuclear", "oil"]).mean()
       
    converter = pd.DataFrame(3.6 , commodity_prices.index, commodity_prices.columns)
    converter["CO2"] = 3.6/1000
    converter.loc["co2"] = 1
    
    commodity_prices = converter*commodity_prices

    to_add = commodity_prices.reindex(["CCGT", "CCGT", "CCGT"])
    to_add.index = ["OCGT", "biomass", "other"]
    to_add.loc[["other"]] = 0
    to_add.loc[["biomass"]] = biomass_price
    to_add.loc[["biomass"], "CO2"] = 0
    commodity_prices = pd.concat([commodity_prices, to_add])
    
    return commodity_prices

In [7]:
def add_renewables():
    
    res = capacity.loc[["onwind","offwind", "solar", "CSP", "ROR"]].unstack().copy()
    res = res.reset_index()
    res.columns = ["bus", "carrier", "p_nom"]
    res.index = res.bus + " " + res.carrier

    res = res[res.p_nom>0]

    vre = ["onwind", "offwind", "solar", "CSP"]

    base_year_res = int(plant_sheets[plant_sheets >= year].subtract(year).idxmin()[2:])
    
    p_max_pu = pd.concat(
        [pd.read_hdf("resources/res_profile.h5", tech).loc[:, str(climate_year), str(base_year_res), :] for tech in vre],
        axis=1
    )

    p_max_pu.columns = vre
    p_max_pu = p_max_pu.unstack(0).stack(0)
    p_max_pu.index = [" ".join(i) for i in p_max_pu.index]
    p_max_pu.columns = snapshots
    p_max_pu = p_max_pu.T

    p_max_pu = pd.concat(
        [p_max_pu, inflows[res.filter(like="ROR", axis=0).index].div(res.filter(like="ROR", axis=0).p_nom)],
        axis=1
    )

    n.madd(
        "Generator",
        res.index,
        **res,
        p_max_pu = p_max_pu[res.index],
        invest_status="policy"
    )

In [8]:
def add_dispatchables():
    
    plants = dispatchable_plants.query("(entry <= @year) & exit > @year")

    plants["marginal_cost"] = (
        commodity_prices
        .reindex(plants.carrier)[base_year].div(
            plants.efficiency.values,
        ).add(
            (
                commodity_prices.reindex(
                    plants.carrier)["CO2"]
                .multiply(
                    commodity_prices.loc["co2", base_year]
                )
            )
        ).add(plants.var_om.values).values
    )

    n.madd(
        "Generator",
        plants.index,
        **plants,
    )

In [9]:
def group_luxembourg(demand, links):
    
    demand_grouper = pd.Series(demand.columns, demand.columns)
    demand_grouper.loc[demand_grouper.index.str[:2] == "LU"] = "LUG1"
    demand = demand.groupby(demand_grouper, axis=1).sum()

    links.loc[links.bus0.str[:2] == "LU", "bus0"] = "LUG1"
    links.loc[links.bus1.str[:2] == "LU", "bus1"] = "LUG1"
    
    return demand, links

In [10]:
def add_dsr():
    
    p_nom_dsr.columns = ["band " + i.split(" ")[2] for i in p_nom_dsr.columns]

    marginal_cost_dsr.columns = ["band " + i.split(" ")[8] for i in marginal_cost_dsr.columns]

    dsr = pd.concat([p_nom_dsr.stack(), marginal_cost_dsr.stack()],axis=1)

    dsr = dsr.loc[:, base_year, :]

    dsr = dsr.set_index((i[0] + " dsr " + i[1] for i in dsr.index), append=True).reset_index([0,1]).drop("level_1",axis=1)

    dsr.columns = ["bus", "p_nom", "marginal_cost"]
    
    dsr["carrier"] = "dsr"

    n.madd(
        "Generator",
        dsr.index,
        **dsr,
        invest_status = "existing"
    )

In [11]:
def set_investment_bounds():
    
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_min"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]

In [12]:
commodity_prices_raw = pd.read_excel("data/Fuel and CO2 prices_ERAA2023.xlsx", index_col = 0)

In [13]:
year = 2029

In [14]:
climate_year = 1990

In [15]:
years = range(2025, 2034)

In [16]:
save_path = "resources/networks/base/cy" + str(climate_year) + "_ty" + str(year) + ".nc"

In [17]:
biomass_price = 7 # make adjustable in config

In [18]:
inflows_raw = pd.read_hdf("resources/inflow.h5", "inflow")

In [19]:
excel_file = pd.ExcelFile("data/pemmdb.xlsx")

In [20]:
snapshots = pd.date_range(start="2010-01-01", freq="h", periods=8760)

In [21]:
plant_sheets = [i for i in excel_file.sheet_names if "TY" in i]
plant_sheets = pd.Series([int(i[2:]) for i in plant_sheets], plant_sheets )

In [22]:
sheet = plant_sheets[plant_sheets <= year].subtract(year).idxmax()

In [23]:
base_year = int(sheet[2:])

In [24]:
distributed_resources = ["onwind", "offwind", "CSP","solar", "battery"]

In [25]:
demand = pd.read_hdf("resources/demand.h5")
demand = demand.loc[:, climate_year, :, str(base_year), :].unstack(1)
demand.index = snapshots
demand.drop("TR00",axis=1, inplace=True)

In [26]:
links = pd.read_hdf("resources/ntcs.h5", "p_nom")
links_p_max_pu = pd.read_hdf("resources/ntcs.h5", "p_max_pu")
links = links[base_year].unstack(1)
links_p_max_pu = links_p_max_pu[base_year].unstack(2)
links_p_max_pu.index = snapshots

In [27]:
links.dropna(inplace=True)
links_p_max_pu.dropna(axis=1, inplace=True)

In [28]:
links["carrier"] = [i[-2:] for i in links.index]

In [29]:
demand, links = group_luxembourg(demand, links)

In [30]:
commodity_prices = prepare_commodity_prices(commodity_prices_raw)

In [31]:
dispatchable_plants = pd.read_hdf("resources/dispatchable_capacities.h5")

In [32]:
technology_data = pd.read_csv("../technology-data/outputs/costs_2025.csv", index_col=[0,1])

In [33]:
n = pypsa.Network()
n.set_snapshots(snapshots)

In [34]:
capacity = build_capacity_table()

In [35]:
p_nom_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, :8]
marginal_cost_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, 8:16]

In [36]:
buses = (
    capacity.sum()[capacity.sum() >0].index
    .union(demand.columns)
    .union(links.bus0.unique())
    .union(links.bus1.unique())
)

In [37]:
n.madd(
    "Bus", 
    buses, 
    carrier = "electricity", 
    country = buses.str[:2]
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BEOF', 'BG00', 'CH00', 'CY00', 'CZ00',
       'DE00', 'DEKF', 'DKBH', 'DKE1', 'DKKF', 'DKW1', 'EE00', 'ES00', 'FI00',
       'FR00', 'GR00', 'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS',
       'ITN1', 'ITS1', 'ITSA', 'ITSI', 'ITVI', 'LT00', 'LUG1', 'LV00', 'ME00',
       'MK00', 'MT00', 'NL00', 'NLLL', 'NOM1', 'NON1', 'NOS0', 'PL00', 'PLE0',
       'PLI0', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04', 'SI00',
       'SK00', 'UK00', 'UKNI'],
      dtype='object')

In [38]:
n.madd(
    "Load", 
    demand.columns,
    bus=demand.columns,
    p_set = demand
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BG00', 'CH00', 'CY00', 'CZ00', 'DE00',
       'DKE1', 'DKW1', 'EE00', 'ES00', 'FI00', 'FR00', 'GR00', 'GR03', 'HR00',
       'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI',
       'LT00', 'LUG1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00', 'NOM1', 'NON1',
       'NOS0', 'PL00', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04',
       'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object', name='zone')

In [39]:
inflows = build_inflows(inflows_raw)

In [40]:
add_existing_storage()

In [41]:
add_dispatchables()

/tmp/ipykernel_2335640/1052378219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants["marginal_cost"] = (


In [42]:
add_renewables()

In [43]:
add_dsr()

In [44]:
set_investment_bounds()

In [45]:
n.madd(
    "Link",
    links.index,
    bus0 = links.bus0,
    bus1 = links.bus1,
    p_nom = links.p_nom,
    p_max_pu = links_p_max_pu,
    carrier = links.carrier
)

Index(['AL00-GR00-AC', 'AL00-ME00-AC', 'AL00-MK00-AC', 'AL00-RS00-AC',
       'AT00-CH00-AC', 'AT00-CZ00-AC', 'AT00-DE00-AC', 'AT00-HU00-AC',
       'AT00-ITN1-AC', 'AT00-SI00-AC',
       ...
       'UK00-BE00-DC', 'UK00-DE00-DC', 'UK00-DKW1-DC', 'UK00-FR00-DC',
       'UK00-IE00-DC', 'UK00-NL00-DC', 'UK00-NOS0-DC', 'UK00-UKNI-DC',
       'UKNI-IE00-AC', 'UKNI-UK00-DC'],
      dtype='object', length=212)

In [46]:
n.madd(
    "Generator",
    n.buses.index + " load-shedding",
    carrier = "load-shedding",
    bus = n.buses.index,
    p_nom=1e5,
    marginal_cost = 10e3,
)

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BEOF load-shedding', 'BG00 load-shedding',
       'CH00 load-shedding', 'CY00 load-shedding', 'CZ00 load-shedding',
       'DE00 load-shedding', 'DEKF load-shedding', 'DKBH load-shedding',
       'DKE1 load-shedding', 'DKKF load-shedding', 'DKW1 load-shedding',
       'EE00 load-shedding', 'ES00 load-shedding', 'FI00 load-shedding',
       'FR00 load-shedding', 'GR00 load-shedding', 'GR03 load-shedding',
       'HR00 load-shedding', 'HU00 load-shedding', 'IE00 load-shedding',
       'ITCA load-shedding', 'ITCN load-shedding', 'ITCS load-shedding',
       'ITN1 load-shedding', 'ITS1 load-shedding', 'ITSA load-shedding',
       'ITSI load-shedding', 'ITVI load-shedding', 'LT00 load-shedding',
       'LUG1 load-shedding', 'LV00 load-shedding', 'ME00 load-shedding',
       'MK00 load-shedding', 'MT00 load-shedding', 'NL00 load-shedding',
       'NLLL load-shedding', 'NOM1 load-shedding', 

In [49]:
n.storage_units.efficiency_store

StorageUnit
AL00 reservoir    1.0
AT00 PHS          1.0
AT00 reservoir    1.0
AT00 battery      1.0
BA00 reservoir    1.0
                 ... 
SK00 reservoir    1.0
SK00 battery      1.0
UK00 PHS          1.0
UK00 battery      1.0
UKNI battery      1.0
Name: efficiency_store, Length: 89, dtype: float64

In [1]:
dirname = os.path.dirname(save_path)  

if not os.path.exists(dirname):
    os.makedirs(dirname)

NameError: name 'os' is not defined

In [ ]:
n.optimize(snapshots=n.snapshots[:24], solver_name="highs")